In [1]:
# =============================================================================
# FASE 1: IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN
# =============================================================================
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
from fredapi import Fred
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [2]:
# --- Configuración del Proyecto ---
# Activos a analizar
tickers_activos = ["SPY", "AAPL", "MSFT", "GOOGL", "NVDA", "AMZN", "COST", "WMT", "MCD", "JPM", "BAC", "V", "JNJ", "PFE", "UNH", "CAT", "BA", "XOM", "CVX", "SQM", "BHP"]
ticker_bono_fred = 'DGS10' # Tasa del Tesoro de EEUU a 10 años

# Fechas para el análisis histórico
end_date = dt.date.today()
start_date = end_date - dt.timedelta(days=5*365) # 5 años de datos

# Parámetros de la simulación
num_portfolios = 25000

# Configuración para la base de datos (simulada con CSV en este ejemplo)
load_dotenv()

db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
key = os.getenv("fred")
fred = Fred(api_key = key)

db_engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')


Acciones

```sql
CREATE TABLE datos_activos (
    fecha DATE NOT NULL,
    ticker VARCHAR(10) NOT NULL,
    precio_cierre NUMERIC,
    volumen BIGINT,
    PRIMARY KEY (fecha, ticker) -- Clave primaria compuesta
);
Bonos

CREATE TABLE datos_bono (
    fecha DATE PRIMARY KEY,
    tasa DECIMAL
);

In [ ]:
datos_activos_raw = yf.download(tickers_activos, start=start_date, end=end_date, auto_adjust = True)
datos_activos_raw

[********************* 43%                       ]  9 of 21 completed

In [ ]:
datos_activos_raw = datos_activos_raw.drop(columns = ["High", "Low", "Open"], axis = 1)
datos_activos_raw

In [ ]:
# Reestructuramos el DataFrame para que sea "largo"
df_activos = datos_activos_raw.stack(future_stack = True, level = 1).reset_index()
df_activos

In [ ]:
df_activos.rename(columns={
    "Date": "fecha",
    "Ticker": "ticker",
    "Volume": "volumen",
    "Close": "precio_cierre"
}, inplace=True)

In [ ]:
df_bonos = fred.get_series(ticker_bono_fred, observation_start = start_date, observation_end = end_date).ffill()
df_bonos = {"fecha": df_bonos.index, "tasa": df_bonos}

In [ ]:
df_bonos = pd.DataFrame(data = df_bonos)
df_bonos

In [ ]:
try:
    with db_engine.connect() as connection:
        # Cargar acciones
        df_activos.to_sql('datos_activos', con=connection, if_exists='replace', index=False)
        print("✅ Datos de acciones cargados exitosamente.")
        
        # Cargar bonos
        df_bonos.to_sql('datos_bonos', con=connection, if_exists='replace', index=False)
        print("✅ Datos de bonos cargados exitosamente.")
except Exception as e:
    print(f"🚨 Error al cargar datos a la base de datos: {e}")


```sql
SELECT
    act.fecha,
    act.ticker,
    act.precio_cierre,
	COALESCE(
    	(precio_cierre / LAG(precio_cierre, 1) OVER (PARTITION BY ticker ORDER BY act.fecha)) - 1,
    	0
	) AS retorno_diario,   	
	ROUND (CAST ((bon.tasa / 100) AS DECIMAL), 4) as tasa_bono,
	act.volumen
FROM
    datos_activos act
LEFT JOIN
    datos_bonos bon ON act.fecha = bon.fecha
ORDER BY
    act.fecha, act.ticker;
```